### 01. 모듈 import / AP몰 접속 / 오늘 하루 다시보지 않기 / brand name 검색

In [309]:
########## 모듈 import ##########
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver as wd
import time

driver = wd.Chrome(executable_path='/Users/hyunginchoi/Desktop/com/python/chromedriver')



########## url과 brand 설정 ##########
ap_url = 'https://www.amorepacificmall.com'
brand_name = '이니스프리'



########## url 접속 후 brand name 검색 ##########
driver.get(ap_url)

driver.find_element_by_class_name('layer_close').click()

driver.find_element_by_id('query_str').send_keys(brand_name)
driver.find_element_by_class_name('btn_search').click()
time.sleep(1)

### 02. 제품 ID 리스트에 저장 / 페이지 이동 반복 / 페이지가 끝나면 except

In [231]:
obj_id = []
try:
    while True:
        ########## 각 제품의 id get ##########
        prd_cell = driver.find_elements_by_class_name('item.item-apply')
        for prd in prd_cell:
            obj_id.append(prd.find_element_by_css_selector('a').get_attribute('href')[-5:])
        
        
        ########## 다음 페이지로 이동 ##########
        driver.find_element_by_css_selector('span + a').click()
        time.sleep(0.3)

########## 다음 페이지가 없을 때 발생하는 에러 ##########
except NoSuchElementException as e:
    print('마지막 페이지입니다')

마지막 페이지입니다


### 03. 데이터 저장할 text file 생성

In [314]:
prd_info = open("apmall_prd_info.txt", "w")
prd_reviews = open("apmall_prd_reviews.txt", "w")

### 04. 각 제품 url에 접속 / 정보 크롤링 / write

In [315]:
k = 1
for i in obj_id:
    
    ########## 얻은 제품 id로 각 제품 url에 접속 ##########
    driver.get(ap_url + '/kr/ko/product/detail?onlineProdSn=' + i)
    time.sleep(0.5)
    
    ########## name, price, review count, point ##########
    prd_name = driver.find_element_by_class_name('product_name').text
    prd_price = driver.find_element_by_class_name('product_price').find_element_by_class_name('num').text
    review_count = driver.find_element_by_class_name('review.visibleReview').find_element_by_class_name('txt').text[3:]
    point = driver.find_element_by_css_selector('div > div.detail_head > div > div.product_info > div.purchase_benefit > p').text
    
    shipping_btn = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(1) > dt > button').click()
    time.sleep(0.1)
    shipping = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(1) > dd > div > p').text
    
    purchase_benefit_btn = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(2) > dt > button').click()
    time.sleep(0.1)
    purchase_benefits = driver.find_elements_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(2) > dd > div > ul > li')
    benefit_total = ''
    for benefit in purchase_benefits:
        span = benefit.find_element_by_css_selector('span').text
        em = '(' + benefit.find_element_by_css_selector('em').text + ')'
        benefit_total += span + em + " "
   

    ########## 리뷰 200개 이상이면 200으로 표현 ##########
    if ',' in review_count or int(review_count) > 200:
        review_count = '200'
    

    
    ########## 리뷰 200개가 보일 때까지 '더보기' 클릭 ##########
    review_click = driver.find_element_by_css_selector('#ap_container > div > div.detail_body.ui_tab.＠tabs-apply > div.tab_menu > ul > li:nth-child(2) > button').click()
    try:
        if int(review_count) < 200:
            while True:
                driver.find_element_by_class_name('btn_list_more').click()
                time.sleep(1)
        else:
            num = 0
            while num < 20:
                driver.find_element_by_class_name('btn_list_more').click()
                time.sleep(1)
                num += 1
    except:
        pass
    
    
    ########## date, rating, user_id, text ##########
    review_box = driver.find_elements_by_class_name('review_box')
    j = 1
    total_rating = 0
    for review in review_box:
        date = review.find_element_by_class_name('rating').find_element_by_class_name('date').text
        rating = review.find_element_by_class_name('rating').find_element_by_css_selector('span:nth-child(1)').get_attribute('class')[-1]
        user_id = review.find_element_by_class_name('name').text
        text = review.find_element_by_class_name('ellipsis.line5').text
        total_rating += int(rating)
        prd_reviews.write(str(k) + " | " + str(j) + " | " + date + " | " + rating + " | " + user_id + " | " + text + "\n")
        j += 1
    prd_reviews.write("\n")    
    
    if review_count == '0':
        rating_avg = '    '
    else:
        rating_avg = str("%.2f" % (total_rating / int(review_count)))

            
    prd_info.write(str(k) + " | " + prd_name + " | " + prd_price + " | " + review_count + " | " + rating_avg + " | " + point[:9] + " | " + shipping_cond[9:28] + " | " + benefit_total + "\n")
    print(str(k) + " 끝")
    k += 1

prd_info.close()
prd_reviews.close()

1 끝
2 끝
3 끝
4 끝
5 끝
6 끝
7 끝
8 끝
9 끝
10 끝
11 끝
12 끝
13 끝
14 끝
15 끝
16 끝
17 끝
18 끝
19 끝
20 끝
21 끝
22 끝
23 끝
24 끝
25 끝
26 끝
27 끝
28 끝
29 끝
30 끝
31 끝
32 끝
33 끝
34 끝
35 끝
36 끝
37 끝
38 끝
39 끝
40 끝
41 끝
42 끝
43 끝
44 끝
45 끝
46 끝
47 끝
48 끝
49 끝
50 끝
51 끝
52 끝
53 끝
54 끝
55 끝
56 끝
57 끝
58 끝
59 끝
60 끝
61 끝
62 끝
63 끝
64 끝
65 끝
66 끝
67 끝
68 끝
69 끝
70 끝
71 끝
72 끝
73 끝
74 끝
75 끝
76 끝
77 끝
78 끝
79 끝
80 끝
81 끝
82 끝
83 끝
84 끝
85 끝
86 끝
87 끝
88 끝
89 끝
90 끝
91 끝
92 끝
93 끝
94 끝
95 끝
96 끝
97 끝
98 끝
99 끝
100 끝
101 끝
102 끝
103 끝
104 끝
105 끝
106 끝
107 끝
108 끝
109 끝
110 끝
111 끝
112 끝
113 끝
114 끝
115 끝
116 끝
117 끝
118 끝
119 끝
120 끝
121 끝
122 끝
123 끝
124 끝
125 끝
126 끝
127 끝
128 끝
129 끝
130 끝
131 끝
132 끝
133 끝
134 끝
135 끝
136 끝
137 끝
138 끝
139 끝
140 끝
141 끝
142 끝
143 끝
144 끝
145 끝
146 끝
147 끝
148 끝
149 끝
150 끝
151 끝
152 끝
153 끝
154 끝
155 끝
156 끝
157 끝
158 끝
159 끝
160 끝
161 끝
162 끝
163 끝
164 끝
165 끝
166 끝
167 끝
168 끝
169 끝
170 끝
171 끝
172 끝
173 끝
174 끝
175 끝
176 끝
177 끝
178 끝
179 끝
180 끝
181 끝
182 끝
183 끝
184 끝
185 

In [316]:
driver.close()